In [1]:
from configparser import ConfigParser
import pandas as pd
import numpy as np
from pathlib import Path
import FuzzyMatch as fm
import DatafileIO as dio
import ComposeFunction as cf
import ConfigReader as cr

In [2]:
import DECAF

In [3]:
input_file = "tests/data/fuzzy_match_companies.xlsx"
col1="reference_name"
col2="new_name"

In [4]:
DECAF.fuzzy_match_companies(input_file, col1, col2)